## サンプル集
相関係数を求めるコードなど、いくつか使えそうなものを載せておきます。
- 相関係数を求める
- 片方でのみ発現した遺伝子に絞る
- 葉緑体の遺伝子にヒットしたかどうか
- 遺伝子のBLAST検索結果を表の時点で全部見たい(DEGs_BLAST.txtで遺伝子名検索が煩わしい時に)

In [ ]:
# データの読み込み
# ファイル名
RES_FILE = 'DE_result.txt'  # 遺伝子発現量・比較結果
BST_FILE = 'DEGs_BLAST.txt' # 詳細情報リスト（BLAST結果）
GO_FILE  = 'DEGs_GO.txt'    # GO_IDリスト

# パッケージやツールをインポート
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 読み込み
RES = pd.read_csv(RES_FILE, sep='\t', header=0,  index_col=0)
BST = pd.read_csv(BST_FILE, sep='\t', header=0,  index_col=0)
GO  = pd.read_csv(GO_FILE, sep='\t', header=-1, index_col=0, names=['GO'])

### 相関係数を求める
まずは図として、反復間のデータの相関を見てみる。<br>
極端に高い値などが含まれていると、それにつられて相関係数が高くなっている場合もあるので、<br>
その場合はその値を除いた上で作図・相関係数の計算を行ってみる。

In [ ]:
# まずは散布図で見てみる。緑のサンプル間での相関の様子
plt.scatter(RES["green_1"], RES["green_2"])
plt.xlabel("green_1")
plt.ylabel("green_2")
plt.show()

# 相関係数を求める
import scipy.stats
PCC = scipy.stats.pearsonr(RES["green_1"], RES["green_2"]) # scipy.stats.pearsonr(x, y)でxとyの相関係数が求まります。
print("green_1とgreen_2の相関係数は", PCC[0])

In [ ]:
# まずは散布図で見てみる。白色のサンプル間での相関の様子
plt.scatter(RES["white_1"], RES["white_2"])
plt.xlabel("white_1")
plt.ylabel("white_2")
plt.show()

# 相関係数を求める
import scipy.stats
PCC = scipy.stats.pearsonr(RES["white_1"], RES["white_2"])
print("white_1とwhite_2の相関係数は", PCC[0])

In [ ]:
sorted_RES = RES.sort_values('green_1', ascending=False) # green_1の高い順に並べる
removed_1st_RES = sorted_RES[1:] # 一番高いものの次からのデータセットを取得
plt.scatter(removed_1st_RES["green_1"], removed_1st_RES["green_2"])
plt.xlabel("green_1")
plt.ylabel("green_2")
plt.show()

# 相関係数を求める
import scipy.stats
PCC = scipy.stats.pearsonr(removed_1st_RES["green_1"], removed_1st_RES["green_2"])
print("極端な値を除いたgreen_1とgreen_2の相関係数は", PCC[0])

In [ ]:
sorted_RES = RES.sort_values('white_1', ascending=False) # white_1の高い順に並べる
removed_1st_RES = sorted_RES[1:] # 一番高いものの次からのデータセットを取得
plt.scatter(removed_1st_RES["white_1"], removed_1st_RES["white_2"])
plt.xlabel("white_1")
plt.ylabel("white_2")
plt.show()

# 相関係数を求める
import scipy.stats
PCC = scipy.stats.pearsonr(removed_1st_RES["white_1"], removed_1st_RES["white_2"])
print("極端な値を除いたwhite_1とwhite_2の相関係数は", PCC[0])

## 片方でのみ発現した遺伝子に絞る
whiteのみで発現している遺伝子やgreenのみで発現している遺伝子。<br>
また、whiteのみで発現している遺伝子の中で、BLAST検索であるキーワードが含まれていたものに絞った遺伝子など。<br>

In [ ]:
# whiteでのみ発現している遺伝子を表示
RES[(RES["green_1"] == 0) & (RES["green_2"] == 0)]

In [ ]:
# greenでのみ発現している遺伝子を表示
RES[(RES["white_1"] == 0) & (RES["white_2"] == 0)]

### 片方で発現しており、かつ、BLAST結果に検索ワードを含む遺伝子を表示

In [ ]:
# 検索ワード
KEYWORD = 'chloro'

# 発現量に差があった遺伝子
sub = RES[RES['FDR']<0.05]

# サンプルwhite or greenでのみ発現していた遺伝子
subA = sub[(sub["green_1"] == 0) & (sub["green_2"] == 0)]
subB = sub[(sub["white_1"] == 0) & (sub["white_2"] == 0)]

# サンプルwhiteでのみ発現した遺伝子のBLAST情報
subA_BST = pd.merge(subA, BST, left_index=True, right_index=True, how='left')
subA_BST[ subA_BST['sprot_Top_BLASTX_hit'].str.contains(KEYWORD) ]

In [ ]:
# サンプルgreenでのみ発現した遺伝子のBLAST情報
subB_BST = pd.merge(subB, BST, left_index=True, right_index=True, how='left')
subB_BST[ subB_BST['sprot_Top_BLASTX_hit'].str.contains(KEYWORD) ]

## 葉緑体の遺伝子にヒットしたかどうか

In [ ]:
# 葉緑体遺伝子にヒットした遺伝子のみ表示
RES[RES["chloroplast"] != "."] # chroloplastの列で"."となっていないものを表示

In [ ]:
# 発現量に差があった遺伝子
sub = RES[RES['FDR']<0.05]

# サンプルwhite or greenでのみ発現していた遺伝子
subA = sub[(sub["green_1"] == 0) & (sub["green_2"] == 0)]
subB = sub[(sub["white_1"] == 0) & (sub["white_2"] == 0)]

# whiteでのみ発現し、葉緑体にヒットした遺伝子
subA[subA["chloroplast"] != "."]

In [ ]:
# greenでのみ発現し、葉緑体にヒットした遺伝子
subB[subB["chloroplast"] != "."]

## 遺伝子のBLAST検索結果を表の時点で全部見たい
遺伝子名まで絞り込めたら、BLASTの結果を見たいと思いますが、DEGs_BLAST.txtで遺伝子名で検索して結果を見るというのは煩わしいかと思います。<br>
そういった場合に、pandasの設定を変更し、省略されていた全文を表示させることができます。<br>
```python
pd.set_option("display.max_colwidth", 文字数)
```
と設定することで、表示できる文字数を増やすことができます。

In [ ]:
pd.set_option("display.max_colwidth", 50)
# 調べたい遺伝子名
gene_name = "TRINITY_DN21887_c2_g6"

data = BST[BST.index == gene_name]
print("普通に表示した場合…")
print(data["sprot_Top_BLASTX_hit"], "\n")

# 表示できる文字数を1000文字に拡張すると(最初は50)
pd.set_option("display.max_colwidth", 2000) # 2000文字に拡張するコード
print("表示文字数を増やすと…")
print(data["sprot_Top_BLASTX_hit"])

In [ ]:
# whiteでのみ発現している遺伝子を表示文字数を増やして表示してみると
pd.set_option("display.max_colwidth", 2000) # 2000文字に拡張するコード

# 検索ワード
KEYWORD = 'chloro'

# 発現量に差があった遺伝子
sub = RES[RES['FDR']<0.05]

# サンプルwhite or greenでのみ発現していた遺伝子
subA = sub[(sub["green_1"] == 0) & (sub["green_2"] == 0)]

# サンプルwhiteでのみ発現した遺伝子のBLAST情報で、検索ワードを含むもの
subA_BST = pd.merge(subA, BST, left_index=True, right_index=True, how='left')
subA_BST[ subA_BST['sprot_Top_BLASTX_hit'].str.contains(KEYWORD) ]

In [ ]:
# 元に戻す場合はこれを動かす
pd.set_option("display.max_colwidth", 50)